# crawler_naver blog

In [ ]:
pwd

### Step 0. 준비

In [ ]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [ ]:
# python 버전 확인
!python --version

In [ ]:
# 판다스 버전 확인
pd.__version__

### Step 1. 크롤링할 블로그 url 수집

In [ ]:
# 검색어
keyword1 = input("1.크롤링할 키워드를 입력하세요: ")
# keyword2 = input("2.제외할 첫번째 키워드를 입력하세요: ")
# keyword3 = input("3.제외할 두번째 키워드를 입력하세요: ")

In [ ]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"G:\내 드라이브\exe\chromedriver.exe")

# 사이트 주소
driver.get("http://www.naver.com")
time.sleep(2)

In [ ]:
# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click()
time.sleep(0.5)

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector(".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

In [ ]:
# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

In [ ]:
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

In [ ]:
url_list

In [ ]:
title_list

In [ ]:
df = pd.DataFrame({'url':url_list, 'title':title_list})
df

In [ ]:
# url_list 저장
df.to_excel("naver_blog_url.xlsx")

### Step 2. 블로그 내용 크롤링

In [ ]:
import sys
import os

import pandas as pd
import numpy as np

In [ ]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("naver_blog_url.xlsx")

In [ ]:
num_list = len(url_load)

print(num_list)
url_load

In [ ]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    driver = webdriver.Chrome(r"G:\내 드라이브\exe\chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                        
        tit = driver.find_element_by_css_selector(overlays)
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        date = driver.find_element_by_css_selector(overlays)
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링 성공하면 글 제목을 출력
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간 저장 (없어도됨)
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

In [ ]:
dict[0]

In [ ]:
# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

In [ ]:
# 엑셀 저장
result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')